In [1]:
import torch
from transformers import AutoTokenizer
import math
import matplotlib
from transformers import AutoTokenizer
import torch

/mnt/d/repos/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Tools

In [2]:
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

In [3]:
def to_tensor(sentence):
    tokens = tokenizer(sentence)
    return torch.tensor([tokens["input_ids"]])

In [4]:
def get_attention_map(model, x):
    attn_maps = []

    # Embed
    B, T = x.size()
    mask = (x != 0)
    x = model.emb_static(x) + model.emb_pos(torch.arange(0, T))
    
    for block in model.blocks:
        # Get the attention map
        B, T, C = x.size()
        qkv = block.attn.attn_matrix(x)
        q, k, v = qkv.split(block.attn.n_embd, dim=2)
        k = k.view(B, T, block.attn.n_head, C // block.attn.n_head).transpose(1, 2)
        q = q.view(B, T, block.attn.n_head, C // block.attn.n_head).transpose(1, 2)
        v = v.view(B, T, block.attn.n_head, C // block.attn.n_head).transpose(1, 2)
        scale_factor = 1 / math.sqrt(q.size(-1))
        attn_weight = q @ k.transpose(-2, -1) * scale_factor
        attn_weight = torch.softmax(attn_weight, dim=-1)
        attn_maps.append(attn_weight)
        
        # Pass through block
        x = block(x, mask, "cpu")
        
    return torch.cat(attn_maps)

In [5]:
def colorize(words, color_array):
    # words is a list of words
    # color_array is an array of numbers between 0 and 1 of length equal to words
    cmap = matplotlib.cm.get_cmap('Greens')
    template = '<span class="barcode" style="color: black; background-color: {};">{}</span>'
    colored_string = ''
    for word, color in zip(words, color_array):
        color = matplotlib.colors.rgb2hex(cmap(color)[:3])
        colored_string += template.format(color, '&nbsp;' + word + '&nbsp;')
    return colored_string

In [6]:
def compare_models(attn_maps1, attn_maps2, sentence, selected):
    # Average attention scores over the sequence
    avg_attention_scores1 = attn_maps1[-1, :, 0, :].mean(dim=[0]).detach().numpy()
    avg_attention_scores2 = attn_maps2[-1, :, 0, :].mean(dim=[0]).detach().numpy()

    # Normalize the attention scores
    normalized_attention_scores1 = (
        (avg_attention_scores1 - avg_attention_scores1.min())
        / (avg_attention_scores1.max() - avg_attention_scores1.min())
    )
    normalized_attention_scores2 = (
        (avg_attention_scores2 - avg_attention_scores2.min())
        / (avg_attention_scores2.max() - avg_attention_scores2.min())
    )

    # Tokenize the text for display
    tokens = tokenizer.convert_ids_to_tokens(tokenizer(sentence)["input_ids"])
    for word in selected:
        if word not in tokens:
            return False, None, None, None, None
    indices = [idx for idx in range(len(tokens)) if tokens[idx] in selected]

    # Remove special tokens like [CLS] and [SEP]
    tokens = tokens[1:-1]
    normalized_attention_scores1 = normalized_attention_scores1[1:-1]
    normalized_attention_scores2 = normalized_attention_scores2[1:-1]

    colored_string1 = colorize(tokens, normalized_attention_scores1)
    colored_string2 = colorize(tokens, normalized_attention_scores2)
    
    return (
        True,
        colored_string1,
        colored_string2,
        [normalized_attention_scores1[idx-1] for idx in indices],
        [normalized_attention_scores2[idx-1] for idx in indices]
    )

# Data

In [7]:
sentences = [
    "The movie was fantastic",
    "I hated the movie",
    "The plot was boring",
    "I love this movie",
    "The plot was terrible",
    "This movie is great",
    "The scenes were dirty",
    "I'm satisfied with movie",
    "The DVD arrived late",
    "The subtitles work perfectly",
    "The movie was disappointing",
    "I enjoyed the movie",
    "The pacing is unreliable",
    "The cast were friendly",
    "The script is slow",
    "The movie was great",
    "The DVD was poor",
    "The plot was fascinating",
    "The set was sturdy",
    "The cinematography was ruined",
    "The documentary was engaging",
    "The DVD crashes often",
    "The scenes were delicious",
    "The DVD broke down",
    "The scenery was breathtaking",
    "The service was prompt",
    "The plot was predictable",
    "The tickets overpriced",
    "The service was excellent",
    "The projector overheats",
    "The theater is scenic",
    "The projector stopped",
    "The festival was vibrant",
    "The popcorn runs out",
    "The movie was fun",
    "The screening was delayed",
    "The impact was pleasant",
    "The streaming is unstable",
    "The snacks are fresh",
    "The DVD cracked",
    "The theater has selection",
    "The interface is difficult",
    "The cinema is spacious",
    "The equipment broke",
    "The staff are friendly",
    "The seats are uncomfortable",
    "The movie was heavenly",
    "The equipment is outdated",
    "The theater is well-kept",
    "The plot was confusing"
]

In [8]:
good_tokens = [
    set({"fantastic"}),
    set({"hated"}),
    set({"boring"}),
    set({"love"}),
    set({"terrible"}),
    set({"great"}),
    set({"dirty"}),
    set({"satisfied"}),
    set({"late"}),
    set({"perfectly"}),
    set({"disappointing"}),
    set({"excellent"}),
    set({"unreliable"}),
    set({"friendly"}),
    set({"slow"}),
    set({"great"}),
    set({"poor"}),
    set({"fascinating"}),
    set({"sturdy"}),
    set({"ruined"}),
    set({"engaging"}),
    set({"crashes"}),
    set({"delicious"}),
    set({"broke"}),
    set({"breathtaking"}),
    set({"prompt"}),
    set({"predictable"}),
    set({"overpriced"}),
    set({"excellent"}),
    set({"overheats"}),
    set({"scenic"}),
    set({"stopped"}),
    set({"vibrant"}),
    set({"quickly"}),
    set({"fun"}),
    set({"delayed"}),
    set({"pleasant"}),
    set({"unstable"}),
    set({"fresh"}),
    set({"cracked"}),
    set({"selection"}),
    set({"difficult"}),
    set({"spacious"}),
    set({"broke"}),
    set({"friendly"}),
    set({"uncomfortable"}),
    set({"heavenly"}),
    set({"outdated"}),
    set({"happy"}),
    set({"confusing"})
]

In [9]:
positive_review = [
    True,
    False,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    True,
    False,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False
]

# Get Results

In [10]:
result = torch.load("results/1.1/6/1.pt", map_location=torch.device('cpu'))
model_1 = result["model"]
result = torch.load("results/2/6/1.pt", map_location=torch.device('cpu'))
model_2 = result["model"]
colored1 = []
colored2 = []
score_list1 = []
score_list2 = []
good_sentences = []
important = []
review_rating = []

for idx in range(len(sentences)):
    sentence = sentences[idx]
    x = to_tensor(sentence)
    attn_maps_1 = get_attention_map(model_1, x)
    attn_maps_2 = get_attention_map(model_2, x)
    (
        good_parse,
        highlighted1,
        highlighted2,
        scores1,
        scores2,
    ) = compare_models(attn_maps_1, attn_maps_2, sentence, good_tokens[idx])
    if not good_parse:
        continue
    colored1.append(highlighted1)
    colored2.append(highlighted2)
    score_list1.append(scores1[0])
    score_list2.append(scores2[0])
    good_sentences.append(sentence)
    important.append(list(good_tokens[idx])[0])
    review_rating.append(positive_review[idx])

/tmp/ipykernel_1717/4248832018.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  result = torch.load("results/1.1/6/1.pt", map_location=torch.device('cpu'))
/tmp/ipykernel

In [11]:
# Setup for HTML table from W3School and stack overflow
# https://www.w3schools.com/html/html_tables.asp
# https://tex.stackexchange.com/questions/23804/how-to-incorporate-tex-mathematics-into-a-website

string = """<!DOCTYPE html>
<html>
<head>
<style>
table {
  font-family: arial, sans-serif;
  border-collapse: collapse;
  width: 50%;
}

td, th {
  border: 1px solid #dddddd;
  text-align: left;
  padding: 8px;
}

tr:nth-child(even) {
  background-color: #dddddd;
}
</style>
<script type="text/x-mathjax-config">
  MathJax.Hub.Config({tex2jax: {inlineMath: [['$','$'], ['\\(','\\)']]}});
</script>
<script type="text/javascript"
  src="http://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-AMS-MML_HTMLorMML">
</script>
</head>
<body>

<table>
  <tr>
    <th>No.</th>
    <th>Label</th>
    <th>Optimal Token</th>
    <th>$\ell_{1.1}$-MD Token Selection</th>
    <th>GD Token Selection</th>
    <th>Better Selector</th>
  </tr>
"""

<>:5: SyntaxWarning: invalid escape sequence '\e'
<>:5: SyntaxWarning: invalid escape sequence '\e'
/tmp/ipykernel_1717/925046660.py:5: SyntaxWarning: invalid escape sequence '\e'
  string = """<!DOCTYPE html>


In [12]:
for idx in range(31):
    string += f"""
  <tr>
    <td>{idx + 1}</td>
    <td>{"+" if review_rating[idx] else "-"}</td>
    <td>{important[idx]}</td>
    <td>{colored1[idx]}</td>
    <td>{colored2[idx]}</td>
    <td>{"=" if score_list1[idx] == score_list2[idx] else "1.1" if score_list1[idx] > score_list2[idx] else "2"}</td>
  </tr>
"""

In [13]:
string += """</table>

</body>
</html>"""

In [15]:
f = open("results/img/attention_map1.html", "w")
f.write(string)
f.close()

In [16]:
string = """<!DOCTYPE html>
<html>
<head>
<style>
table {
  font-family: arial, sans-serif;
  border-collapse: collapse;
  width: 50%;
}

td, th {
  border: 1px solid #dddddd;
  text-align: left;
  padding: 8px;
}

tr:nth-child(even) {
  background-color: #dddddd;
}
</style>
<script type="text/x-mathjax-config">
  MathJax.Hub.Config({tex2jax: {inlineMath: [['$','$'], ['\\(','\\)']]}});
</script>
<script type="text/javascript"
  src="http://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-AMS-MML_HTMLorMML">
</script>
</head>
<body>

<table>
"""

In [17]:
for idx in range(31, 40):
    string += f"""
  <tr>
    <td>{idx + 1}</td>
    <td>{"+" if review_rating[idx] else "-"}</td>
    <td>{important[idx]}</td>
    <td>{colored1[idx]}</td>
    <td>{colored2[idx]}</td>
    <td>{"=" if score_list1[idx] == score_list2[idx] else "1.1" if score_list1[idx] > score_list2[idx] else "2"}</td>
  </tr>
"""

In [18]:
string += """</table>

</body>
</html>"""

In [19]:
f = open("results/img/attention_map2.html", "w")
f.write(string)
f.close()

In [15]:
print(f"Score for 1.1-MD = {torch.tensor(score_list1).mean().item()}")

Score for 1.1-MD = 0.4387154281139374


In [16]:
print(f"Score for GD = {torch.tensor(score_list2).mean().item()}")

Score for GD = 0.3122487962245941
